# Configuration

In [1]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [0]:
# import des libraries
import numpy as np
import pandas as pd

# Importation et préparation des données

## Revenus

In [3]:
# importation données
url_data = "https://raw.githubusercontent.com/xavierbarbier/openclassrooms/master/projet_7/data/data-projet7.csv"

data = pd.read_csv(url_data,decimal=",")

data.head(5)

,country,year_survey,quantile,nb_quantiles,income,gdpppp
0,ALB,2008,1,100,728.89795,7297.0
1,ALB,2008,2,100,916.66235,7297.0
2,ALB,2008,3,100,1010.91600,7297.0
3,ALB,2008,4,100,1086.90780,7297.0
4,ALB,2008,5,100,1132.69970,7297.0


In [4]:
# format du data frame
data.shape

(11599, 6)

In [5]:
np.unique(data["year_survey"])

array([2004, 2006, 2007, 2008, 2009, 2010, 2011])

In [6]:
len(np.unique(data["country"]))

116

* manque 1 ligne (116 pays * 100 = 11600)

In [7]:
# recherche pay avec erreur
error = data.groupby(["country"]).mean()
error.sort_values(by=['quantile'], inplace=True, ascending=False)
error.head(5)

,year_survey,quantile,nb_quantiles,income,gdpppp
country,,,,,
LTU,2008.0,50.59596,100.0,6641.247634,17571.0
ALB,2008.0,50.50000,100.0,2994.829902,7297.0
MOZ,2008.0,50.50000,100.0,692.480501,773.0
PER,2008.0,50.50000,100.0,3330.533349,7858.0
PAN,2009.0,50.50000,100.0,5135.139376,11767.0


* Lituanie avec erreur.

### LTU

In [8]:
# recherche quantile manquant
ltu = data[data["country"]=="LTU"]

ltu_qt = list(ltu["quantile"])
total_qt = list(range(1,100))

list_difference = []
for item in total_qt:
  if item not in ltu_qt:
    list_difference.append(item)

print(list_difference)

[41]


In [9]:
ltu[ltu["quantile"]==41]

,country,year_survey,quantile,nb_quantiles,income,gdpppp


* Manque quantile 41


In [10]:
# Calcul moyenne quantiles adjacents
qt_40 = ltu.income[ltu["quantile"]==40].values
qt_42 = ltu.income[ltu["quantile"]==42].values
qt_41 = (qt_40+qt_42)/2
qt_41


array([4882.14065])

* Quantile 41 LTU = 4882.14065

In [11]:
# création ligne moyenne quantiles adjacents

ltu_41 = ltu[ltu["quantile"]==40]

ltu_41["quantile"] = 41
ltu_41.at[6239, 'income'] = qt_41

ltu_41

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,country,year_survey,quantile,nb_quantiles,income,gdpppp
6239,LTU,2008,41,100,4882.14065,17571.0


In [0]:
# Ajout ligne manquante
data = pd.concat([data, ltu_41])


In [13]:
# vérification format df
data.shape

(11600, 6)

In [14]:
# vérification valeurs abberantes
error = data.groupby(["country"]).mean()
error.sort_values(by=['quantile'], inplace=True, ascending=False)
error.head(5)

,year_survey,quantile,nb_quantiles,income,gdpppp
country,,,,,
ALB,2008.0,50.5,100.0,2994.829902,7297.0
MOZ,2008.0,50.5,100.0,692.480501,773.0
PER,2008.0,50.5,100.0,3330.533349,7858.0
PAN,2009.0,50.5,100.0,5135.139376,11767.0
PAK,2008.0,50.5,100.0,887.839279,2335.0


* 100 centile par pays = OK

In [15]:
data.tail(5)

,country,year_survey,quantile,nb_quantiles,income,gdpppp
11595,COD,2008,97,100,911.78340,303.19305
11596,COD,2008,98,100,1057.80740,303.19305
11597,COD,2008,99,100,1286.60290,303.19305
11598,COD,2008,100,100,2243.12260,303.19305
6239,LTU,2008,41,100,4882.14065,17571.00000


In [16]:
# rangement données
data.sort_values(by=['country', "quantile"], inplace=True, ascending=True)
data = data.reset_index(drop=True)
data.tail(5)

,country,year_survey,quantile,nb_quantiles,income,gdpppp
11595,ZAF,2008,96,100,24553.568,9602.0
11596,ZAF,2008,97,100,28858.031,9602.0
11597,ZAF,2008,98,100,35750.290,9602.0
11598,ZAF,2008,99,100,46297.316,9602.0
11599,ZAF,2008,100,100,82408.550,9602.0


In [17]:
# recherche valeur nulles
print(pd.isnull(data).sum())

country           0
year_survey       0
quantile          0
nb_quantiles      0
income            0
gdpppp          200
dtype: int64


In [18]:
# identification valeurs nulles
np.unique(data.loc[pd.isna(data["gdpppp"]), :].country)

array(['PSE', 'XKX'], dtype=object)

* 2 pays en Na's : PSE (Palestine) et XKX (Kosovo) 

PSE = 3606,9 (www.theglobaleconomy.com)
XKX = 7249,409 (data.worldbank.org/)

### PSE et XKX

In [19]:
# impuation valeur PSE
data.iloc[9000:9100]["gdpppp"] = 3606.9
data[data["country"]== "PSE"]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,country,year_survey,quantile,nb_quantiles,income,gdpppp
9000,PSE,2009,1,100,195.28990,3606.9
9001,PSE,2009,2,100,264.36533,3606.9
9002,PSE,2009,3,100,301.44672,3606.9
9003,PSE,2009,4,100,329.83392,3606.9
9004,PSE,2009,5,100,348.76495,3606.9
...,...,...,...,...,...,...
9095,PSE,2009,96,100,2763.88480,3606.9
9096,PSE,2009,97,100,3077.83330,3606.9
9097,PSE,2009,98,100,3449.22240,3606.9
9098,PSE,2009,99,100,4165.99700,3606.9


In [20]:
data[data["country"]== "XKX"]

,country,year_survey,quantile,nb_quantiles,income,gdpppp
11300,XKX,2008,1,100,437.89370,NaN
11301,XKX,2008,2,100,508.17133,NaN
11302,XKX,2008,3,100,591.82820,NaN
11303,XKX,2008,4,100,668.00000,NaN
11304,XKX,2008,5,100,730.40220,NaN
...,...,...,...,...,...,...
11395,XKX,2008,96,100,5155.36470,NaN
11396,XKX,2008,97,100,5689.52930,NaN
11397,XKX,2008,98,100,6233.73930,NaN
11398,XKX,2008,99,100,7366.67700,NaN


In [21]:
# imputation valeur XKX
data.iloc[11300:11400]["gdpppp"] = 7249.409
data[data["country"]== "XKX"]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,country,year_survey,quantile,nb_quantiles,income,gdpppp
11300,XKX,2008,1,100,437.89370,7249.409
11301,XKX,2008,2,100,508.17133,7249.409
11302,XKX,2008,3,100,591.82820,7249.409
11303,XKX,2008,4,100,668.00000,7249.409
11304,XKX,2008,5,100,730.40220,7249.409
...,...,...,...,...,...,...
11395,XKX,2008,96,100,5155.36470,7249.409
11396,XKX,2008,97,100,5689.52930,7249.409
11397,XKX,2008,98,100,6233.73930,7249.409
11398,XKX,2008,99,100,7366.67700,7249.409


In [22]:
# recherche valeur nulles
print(pd.isnull(data).sum())

country         0
year_survey     0
quantile        0
nb_quantiles    0
income          0
gdpppp          0
dtype: int64


* Plus aucunes valeurs nulles

In [0]:
data_clean = data.copy()

## Gini World Bank

In [24]:
# importation données
url_gini_wb = "https://raw.githubusercontent.com/xavierbarbier/openclassrooms/master/projet_7/data/API_SI_POV_GINI_DS2_en_csv_v2_988343.csv"

gini_wb = pd.read_csv(url_gini_wb, skiprows= 4)

gini_wb.head(5)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Unnamed: 64
0,Aruba,ABW,GINI index (World Bank estimate),SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,GINI index (World Bank estimate),SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Angola,AGO,GINI index (World Bank estimate),SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.3,NaN,NaN
3,Albania,ALB,GINI index (World Bank estimate),SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.0,NaN,NaN,NaN,NaN,NaN,31.7,NaN,NaN,30.6,NaN,NaN,30.0,NaN,NaN,NaN,29.0,NaN,34.6,32.9,33.7,33.2,NaN,NaN,NaN
4,Andorra,AND,GINI index (World Bank estimate),SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# recherche valeur nulles
print(pd.isnull(gini_wb).sum())

Country Name        0
Country Code        0
Indicator Name      0
Indicator Code      0
1960              264
                 ... 
2016              188
2017              198
2018              236
2019              264
Unnamed: 64       264
Length: 65, dtype: int64


In [26]:
len(gini_wb)

264

* 264 pays

In [27]:
# suppression valeurs nulles
gini_clean = gini_wb.dropna()
gini_clean.head(5)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Unnamed: 64


In [28]:
# nombre pays données complètes
print(len(gini_clean))

0


* 47 pays avec données gini complètes

In [29]:
gini_wb.head(5)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Unnamed: 64
0,Aruba,ABW,GINI index (World Bank estimate),SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,GINI index (World Bank estimate),SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Angola,AGO,GINI index (World Bank estimate),SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.3,NaN,NaN
3,Albania,ALB,GINI index (World Bank estimate),SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.0,NaN,NaN,NaN,NaN,NaN,31.7,NaN,NaN,30.6,NaN,NaN,30.0,NaN,NaN,NaN,29.0,NaN,34.6,32.9,33.7,33.2,NaN,NaN,NaN
4,Andorra,AND,GINI index (World Bank estimate),SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
gini_wb.to_csv("/content/drive/My Drive/DATASETS/gini_wb.csv", index=False)

### calcul gini

In [0]:
#calcul gini

country_gini = []

for i in data.country : 
  inc = data[data.country==i]["income"]
  n = len(inc)
  lorenz = np.cumsum(np.sort(inc)) / inc.sum()
  lorenz = np.append([0],lorenz) # La courbe de Lorenz commence à 0

  AUC = (lorenz.sum() -lorenz[-1]/2 -lorenz[0]/2)/n # Surface sous la courbe de Lorenz. Le premier segment (lorenz[0]) est à moitié en dessous de 0, on le coupe donc en 2, on fait de même pour le dernier segment lorenz[-1] qui est à moitié au dessus de 1.
  S = 0.5 - AUC # surface entre la première bissectrice et le courbe de Lorenz
  gini = round(2*S,2)
  country_gini.append(gini)


In [32]:
# vérification longueur
len(country_gini)

11600

In [33]:
# assignation gini à data
data["gini"] = country_gini
data.head(5)

,country,year_survey,quantile,nb_quantiles,income,gdpppp,gini
0,ALB,2008,1,100,728.89795,7297.0,0.3
1,ALB,2008,2,100,916.66235,7297.0,0.3
2,ALB,2008,3,100,1010.91600,7297.0,0.3
3,ALB,2008,4,100,1086.90780,7297.0,0.3
4,ALB,2008,5,100,1132.69970,7297.0,0.3


In [34]:
# conservation données gini 
country_gini = data[["country","gini"]]
country_gini = pd.DataFrame(country_gini.groupby(["country"]).mean())
country_gini = country_gini.reset_index()
country_gini.head(5)

,country,gini
0,ALB,0.30
1,ARG,0.45
2,ARM,0.26
3,AUT,0.28
4,AZE,0.31


In [0]:
country_gini.to_csv("/content/drive/My Drive/DATASETS/country_gini.csv", index=False)

## Population

In [36]:
# importation données
url_pop = "https://raw.githubusercontent.com/xavierbarbier/openclassrooms/master/projet_7/data/FAOSTAT_data_4-30-2020.csv"

population = pd.read_csv(url_pop)

population.head(5)

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,OA,Séries temporelles annuelles,AFG,Afghanistan,511,Population totale,3010,Population-Estimations,2004,2004,1000 personnes,24726.684,X,Sources internationales sûres,NaN
1,OA,Séries temporelles annuelles,AFG,Afghanistan,511,Population totale,3010,Population-Estimations,2006,2006,1000 personnes,26433.049,X,Sources internationales sûres,NaN
2,OA,Séries temporelles annuelles,AFG,Afghanistan,511,Population totale,3010,Population-Estimations,2007,2007,1000 personnes,27100.536,X,Sources internationales sûres,NaN
3,OA,Séries temporelles annuelles,AFG,Afghanistan,511,Population totale,3010,Population-Estimations,2008,2008,1000 personnes,27722.276,X,Sources internationales sûres,NaN
4,OA,Séries temporelles annuelles,AFG,Afghanistan,511,Population totale,3010,Population-Estimations,2009,2009,1000 personnes,28394.813,X,Sources internationales sûres,NaN


In [37]:
# conversion valeur
population["Valeur"] = round((population["Valeur"]*1000)/1000000,3)

population = population.pivot_table(
    index=["Zone","Code zone"],
    columns = ["Année"], values=["Valeur"])

population = population.reset_index()

# Rangement colonnes
population.columns = ["Zone",'Code zone','2004',"2006","2007","2008","2009","2010","2011"]

population.head(5)

,Zone,Code zone,2004,2006,2007,2008,2009,2010,2011
0,Afghanistan,AFG,24.727,26.433,27.101,27.722,28.395,29.186,30.117
1,Afrique du Sud,ZAF,47.292,48.489,49.120,49.779,50.477,51.217,52.004
2,Albanie,ALB,3.105,3.063,3.034,3.003,2.973,2.948,2.929
3,Algérie,DZA,32.692,33.641,34.167,34.731,35.334,35.977,36.661
4,Allemagne,DEU,81.646,81.472,81.278,81.066,80.900,80.827,80.856


In [38]:
# recherche valeur nulles
print(pd.isnull(population).sum())

Zone         0
Code zone    0
2004         7
2006         6
2007         6
2008         6
2009         6
2010         6
2011         1
dtype: int64


* Nombreuses valeurs manquantes

In [0]:
# imputation par moyenne sur période 2004-2011
population["pop"] = round(population.iloc[:, 2:].mean(axis=1),3)

In [40]:
# recherche valeur nulles
print(pd.isnull(population).sum())

Zone         0
Code zone    0
2004         7
2006         6
2007         6
2008         6
2009         6
2010         6
2011         1
pop          0
dtype: int64


* 0 valeurs nulles sur moyenne population 2004-2011

In [41]:
len(np.unique(population["Zone"]))

233

* 233 pays concernés

In [42]:
population = population [["Zone",	"Code zone","pop"]]
population.head(5)

,Zone,Code zone,pop
0,Afghanistan,AFG,27.669
1,Afrique du Sud,ZAF,49.768
2,Albanie,ALB,3.008
3,Algérie,DZA,34.743
4,Allemagne,DEU,81.149


In [0]:
population.to_csv("/content/drive/My Drive/DATASETS/population.csv", index=False)

In [44]:
# jointure données population sur data
data = pd.merge(data,population,left_on= "country", right_on= "Code zone", how = "left")
data = data[["country","year_survey","quantile","nb_quantiles","income","gdpppp","gini","pop"]]
data.head(5)

,country,year_survey,quantile,nb_quantiles,income,gdpppp,gini,pop
0,ALB,2008,1,100,728.89795,7297.0,0.3,3.008
1,ALB,2008,2,100,916.66235,7297.0,0.3,3.008
2,ALB,2008,3,100,1010.91600,7297.0,0.3,3.008
3,ALB,2008,4,100,1086.90780,7297.0,0.3,3.008
4,ALB,2008,5,100,1132.69970,7297.0,0.3,3.008


In [45]:
# recherche valeur nulles
print(pd.isnull(data).sum())

country           0
year_survey       0
quantile          0
nb_quantiles      0
income            0
gdpppp            0
gini              0
pop             400
dtype: int64


* 400 valeurs manquantes (4 pays)

In [46]:
# identification valeurs manquantes
error = data.loc[pd.isna(data["pop"]), :]
np.unique(list(error["country"]))


array(['CHN', 'SDN', 'TWN', 'XKX'], dtype='<U3')

* population manquantes : 'CHN', 'SDN', 'TWN', 'XKX'

### Valeurs manquantes via FAO

In [47]:
# importation données
url_missing_pop = "https://raw.githubusercontent.com/xavierbarbier/openclassrooms/master/projet_7/data/FAOSTAT_data_missing.csv"

missing_pop = pd.read_csv(url_missing_pop)

# conversion valeur
missing_pop["Valeur"] = round((missing_pop["Valeur"]*1000)/1000000,3)

missing_pop = missing_pop.pivot_table(
    index=["Zone","Code zone"],
    columns = ["Année"], values=["Valeur"])

missing_pop = missing_pop.reset_index()

# Rangement colonnes
missing_pop.columns = ["Zone",'Code zone',"pop"]

# renommage colonnes Soudan
missing_pop.iloc[2,1] = "SDN"

missing_pop.head(5)

,Zone,Code zone,pop
0,Chine,CHN,1383.986
1,"Chine, Taiwan Province de",TWN,23.019
2,Soudan (ex),SDN,41.798


#### CHN/TWN/SDN/XKX

In [0]:
# définition valeurs
chn = missing_pop[missing_pop["Code zone"]=="CHN"]["pop"]
twn = missing_pop[missing_pop["Code zone"]=="TWN"]["pop"]
sdn = missing_pop[missing_pop["Code zone"]=="SDN"]["pop"]
xkx = 1.747 # banque mondiale / google

In [0]:
# imputation valeurs
data['pop'] = data['pop'].fillna(data["country"].map({'CHN':float(chn),'TWN':float(twn),
                'SDN':float(sdn),'XKX':float(xkx)}))

In [50]:
# recherche valeur nulles
print(pd.isnull(data).sum())

error = data.loc[pd.isna(data["pop"]), :]
print(np.unique(list(error["country"])))

country         0
year_survey     0
quantile        0
nb_quantiles    0
income          0
gdpppp          0
gini            0
pop             0
dtype: int64
[]


* 0 Valeurs nulles

In [51]:
data.head(10)

,country,year_survey,quantile,nb_quantiles,income,gdpppp,gini,pop
0,ALB,2008,1,100,728.89795,7297.0,0.3,3.008
1,ALB,2008,2,100,916.66235,7297.0,0.3,3.008
2,ALB,2008,3,100,1010.91600,7297.0,0.3,3.008
3,ALB,2008,4,100,1086.90780,7297.0,0.3,3.008
4,ALB,2008,5,100,1132.69970,7297.0,0.3,3.008
5,ALB,2008,6,100,1171.14120,7297.0,0.3,3.008
6,ALB,2008,7,100,1201.13240,7297.0,0.3,3.008
7,ALB,2008,8,100,1240.89760,7297.0,0.3,3.008
8,ALB,2008,9,100,1285.69140,7297.0,0.3,3.008
9,ALB,2008,10,100,1325.25330,7297.0,0.3,3.008


## Coef élasticité

### WB data (country)

In [52]:
# importation données
url_elast = "https://raw.githubusercontent.com/xavierbarbier/openclassrooms/master/projet_7/data/GDIMMay2018.csv"

elast = pd.read_csv(url_elast)

elast.head(5)

,countryname,wbcode,iso3,region,incgroup2,incgroup4,fragile,survey,year,status,cohort,parent,child,obs,P1,P2,P3,P4,P5,C1,C2,C3,C4,C5,MEANp,MEANc,SDp,SDc,GINIp,GINIc,IGP,NL1,NL2,COR,MAcatM,MAcatC1,Q4_IGpri,BHQ1,BHQ2,BHQ3,BHQ4,Q4BH,Q4child,Delta50,Asher_Q4_IGpri,ThreeGen_obs,ThreeGen_IGPp1,ThreeGen_IGPg1,ThreeGen_IGPgsd,All2125_MEANp,All2125_MEANc,All2125_IGP,All2125_MAcatC1,Cores2125_MEANp,Cores2125_MEANc,Cores2125_IGP,Cores2125_MAcatC1,Shortfall0611_obs,Shortfall0611_IGP,Shortfall1217_obs,Shortfall1217_IGP,IGEincome,S1,S2,S3,MLD_psu
0,Afghanistan,AFG,AFG,South Asia,Developing economies,Low income,1,NRVA,1980,Co-residents only,1980,avg,all,3956.0,0.666125,0.098583,0.170931,0.043031,0.021330,0.511857,0.025805,0.164191,0.183469,0.114678,2.480162,5.582401,3.809782,5.857765,0.728547,0.569981,0.802107,1.078013,-0.025322,0.521676,0.416691,0.414355,0.573748,0.341912,0.325161,0.209395,0.123532,0.162255,0.250311,3.408008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25103.0,0.086197,18054.0,0.345224,NaN,NaN,NaN,NaN,0.1
1,Afghanistan,AFG,AFG,South Asia,Developing economies,Low income,1,NRVA,1980,Co-residents only,1980,avg,daughter,721.0,0.615321,0.099272,0.212776,0.051969,0.020662,0.628481,0.023378,0.099265,0.132322,0.116553,2.989992,4.331731,4.010379,5.902685,0.675526,0.681604,0.848156,0.904866,-0.005200,0.576251,0.284895,0.283075,0.493111,0.434127,0.390918,0.101283,0.073671,0.285185,0.214735,1.859528,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12107.0,0.083271,8538.0,0.389952,NaN,NaN,NaN,NaN,0.1
2,Afghanistan,AFG,AFG,South Asia,Developing economies,Low income,1,NRVA,1980,Co-residents only,1980,avg,son,3235.0,0.678699,0.098412,0.160574,0.040819,0.021495,0.482992,0.026405,0.180261,0.196128,0.114214,2.353979,5.891941,3.748495,5.806141,0.741541,0.541870,0.807674,1.168599,-0.033091,0.521442,0.449310,0.446874,0.599971,0.322116,0.311027,0.232628,0.134229,0.122268,0.259117,3.740587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12996.0,0.089161,9516.0,0.307687,NaN,NaN,NaN,NaN,0.1
3,Afghanistan,AFG,AFG,South Asia,Developing economies,Low income,1,NRVA,1980,Co-residents only,1980,dad,all,4137.0,0.686581,0.047610,0.057309,0.150907,0.057594,0.514562,0.026047,0.165638,0.181551,0.112202,3.756049,5.538275,5.493762,5.842035,0.713742,0.572566,0.514428,0.578723,-0.004792,0.483760,0.354333,0.350244,0.548029,0.334443,0.317131,0.219586,0.128840,0.192434,0.250295,3.538447,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25396.0,0.050447,18387.0,0.218062,NaN,NaN,NaN,NaN,0.1
4,Afghanistan,AFG,AFG,South Asia,Developing economies,Low income,1,NRVA,1980,Co-residents only,1980,dad,daughter,744.0,0.639092,0.052328,0.054086,0.190100,0.064394,0.630354,0.022781,0.103736,0.129553,0.113576,4.520877,4.277768,5.804775,5.869737,0.660039,0.684751,0.491792,0.553255,-0.004494,0.486349,0.235170,0.230126,0.454504,0.429996,0.385429,0.104363,0.080212,0.339629,0.212542,1.974941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12246.0,0.047961,8677.0,0.230909,NaN,NaN,NaN,NaN,0.1


In [53]:
np.unique(list(elast["region"]))

array(['East Asia & Pacific', 'Europe & Central Asia', 'High income',
       'Latin America & Caribbean', 'Middle East & North Africa',
       'South Asia', 'Sub-Saharan Africa'], dtype='<U26')

In [54]:
np.unique(list(elast["year"]))

array([1940, 1950, 1960, 1970, 1980])

In [0]:
# conservation données parent et enfant
elast = elast[elast["parent"]== "avg"]
elast = elast[elast["child"]== "all"]
elast = elast[["countryname", "iso3","region","year","IGEincome"]]

In [56]:
elast.head(5)

,countryname,iso3,region,year,IGEincome
0,Afghanistan,AFG,South Asia,1980,NaN
12,Angola,AGO,Sub-Saharan Africa,1980,NaN
24,Albania,ALB,Europe & Central Asia,1940,NaN
36,Albania,ALB,Europe & Central Asia,1950,NaN
48,Albania,ALB,Europe & Central Asia,1960,NaN


In [0]:
elast_full = elast
elast_full.to_csv("/content/drive/My Drive/DATASETS/elast_full_clean.csv", index=False)

In [58]:
elast.shape

(531, 5)

In [59]:
# recherche valeur nulles
print(pd.isnull(elast).sum())

countryname      0
iso3             0
region           0
year             0
IGEincome      461
dtype: int64


* 461 valeurs manquantes

In [0]:
# calcul moyenne par pays sur durée données
elast = elast.groupby(["iso3","region"]).mean()


In [61]:
elast = elast.reset_index()
elast.head(5)

,iso3,region,year,IGEincome
0,AFG,South Asia,1980,NaN
1,AGO,Sub-Saharan Africa,1980,NaN
2,ALB,Europe & Central Asia,1960,0.815874
3,ARM,Europe & Central Asia,1960,NaN
4,AUS,High income,1960,0.275000


In [62]:
len(elast)

137

In [63]:
error = elast.loc[pd.isna(elast["IGEincome"]), :]
np.unique(list(error["iso3"]))

array(['AFG', 'AGO', 'ARM', 'AZE', 'BFA', 'BGD', 'BGR', 'BTN', 'BWA',
       'CAF', 'CIV', 'CMR', 'COG', 'COM', 'CPV', 'DJI', 'EST', 'FJI',
       'GAB', 'GEO', 'GNB', 'HUN', 'IDN', 'IRN', 'IRQ', 'ISL', 'ISR',
       'KHM', 'KIR', 'KSV', 'LAO', 'LBN', 'LBR', 'LKA', 'LSO', 'LTU',
       'MDA', 'MDV', 'MEX', 'MNE', 'MOZ', 'MRT', 'MUS', 'NAM', 'NER',
       'PHL', 'PNG', 'POL', 'PSE', 'SDN', 'SEN', 'SLE', 'SRB', 'SSD',
       'STP', 'SWZ', 'TCD', 'TGO', 'THA', 'TJK', 'TON', 'TUR', 'TUV',
       'UKR', 'VUT', 'YEM', 'ZMB'], dtype='<U3')

In [64]:
elast[elast["iso3"]=="FRA"]

,iso3,region,year,IGEincome
40,FRA,High income,1960,0.357105


In [0]:
data = pd.merge(data,elast,left_on= "country", right_on= "iso3", how = "left" )

In [66]:
del data["iso3"]
data.head(5)

,country,year_survey,quantile,nb_quantiles,income,gdpppp,gini,pop,region,year,IGEincome
0,ALB,2008,1,100,728.89795,7297.0,0.3,3.008,Europe & Central Asia,1960.0,0.815874
1,ALB,2008,2,100,916.66235,7297.0,0.3,3.008,Europe & Central Asia,1960.0,0.815874
2,ALB,2008,3,100,1010.91600,7297.0,0.3,3.008,Europe & Central Asia,1960.0,0.815874
3,ALB,2008,4,100,1086.90780,7297.0,0.3,3.008,Europe & Central Asia,1960.0,0.815874
4,ALB,2008,5,100,1132.69970,7297.0,0.3,3.008,Europe & Central Asia,1960.0,0.815874


In [67]:
# recherche valeur nulles
print(pd.isnull(data).sum())

country            0
year_survey        0
quantile           0
nb_quantiles       0
income             0
gdpppp             0
gini               0
pop                0
region          1300
year            1300
IGEincome       5300
dtype: int64


In [68]:
data.shape

(11600, 11)

### WB data (region)

In [69]:
# importation données
url_region = "https://raw.githubusercontent.com/xavierbarbier/openclassrooms/master/projet_7/data/GDIMprovinceMay2018.csv"

region = pd.read_csv(url_region)

region = region[["countryname","iso3","region"]]

region.head(5)

,countryname,iso3,region
0,Brazil,BRA,Latin America & Caribbean
1,Brazil,BRA,Latin America & Caribbean
2,Brazil,BRA,Latin America & Caribbean
3,Brazil,BRA,Latin America & Caribbean
4,Brazil,BRA,Latin America & Caribbean


In [70]:
np.unique(list(region["countryname"]))

array(['Brazil', 'China', 'Egypt, Arab Rep.', 'Indonesia', 'Nigeria'],
      dtype='<U16')

* inutile...

### OC data (txt)

East Asia & Pacific', 'Europe & Central Asia', 'High income',
       'Latin America & Caribbean', 'Middle East & North Africa',
       'South Asia', 'Sub-Saharan Africa'

In [0]:
# définition valeurs selon région du fichier txt 
eap =0.4
eca = 0.4
lac = 0.66
mena = 0.66
sa = 0.5
ssa = 0.66

In [0]:
# Imputation valeurs selon régions
data['IGEincome'] = data['IGEincome'].fillna(data["region"].map({'East Asia & Pacific':float(eap),
                'Europe & Central Asia':float(eca),
                'Latin America & Caribbean':float(lac),
                'Middle East & North Africa':float(mena),
                'South Asia':float(sa),
                'Sub-Saharan Africa':float(ssa)}))

In [73]:
# recherche valeur nulles
print(pd.isnull(data).sum())

country            0
year_survey        0
quantile           0
nb_quantiles       0
income             0
gdpppp             0
gini               0
pop                0
region          1300
year            1300
IGEincome       1900
dtype: int64


In [74]:
error = elast.loc[pd.isna(elast["IGEincome"]), :]
np.unique(list(error["iso3"]))

array(['AFG', 'AGO', 'ARM', 'AZE', 'BFA', 'BGD', 'BGR', 'BTN', 'BWA',
       'CAF', 'CIV', 'CMR', 'COG', 'COM', 'CPV', 'DJI', 'EST', 'FJI',
       'GAB', 'GEO', 'GNB', 'HUN', 'IDN', 'IRN', 'IRQ', 'ISL', 'ISR',
       'KHM', 'KIR', 'KSV', 'LAO', 'LBN', 'LBR', 'LKA', 'LSO', 'LTU',
       'MDA', 'MDV', 'MEX', 'MNE', 'MOZ', 'MRT', 'MUS', 'NAM', 'NER',
       'PHL', 'PNG', 'POL', 'PSE', 'SDN', 'SEN', 'SLE', 'SRB', 'SSD',
       'STP', 'SWZ', 'TCD', 'TGO', 'THA', 'TJK', 'TON', 'TUR', 'TUV',
       'UKR', 'VUT', 'YEM', 'ZMB'], dtype='<U3')

* Toujours 6 pays avec NaN's

### Region

In [0]:
data['IGEincome'] = data['IGEincome'].fillna(data.groupby('region')['IGEincome'].transform('mean'))

In [76]:
# recherche valeur nulles
print(pd.isnull(data).sum())

country            0
year_survey        0
quantile           0
nb_quantiles       0
income             0
gdpppp             0
gini               0
pop                0
region          1300
year            1300
IGEincome       1300
dtype: int64


In [77]:
error = data.loc[pd.isna(data["IGEincome"]), :]
np.unique(list(error["country"]))

array(['ARG', 'COD', 'CRI', 'DOM', 'HND', 'LUX', 'NIC', 'PRY', 'SLV',
       'SYR', 'URY', 'VEN', 'XKX'], dtype='<U3')

In [78]:
'East Asia & Pacific'
'Europe & Central Asia'
'Latin America & Caribbean'
'Middle East & North Africa'
'South Asia'
'Sub-Saharan Africa'
"High income"

'High income'

In [0]:
# Imputation valeurs régions
data["region"] = data["region"].fillna(data["country"].map({'ARG':'Latin America & Caribbean',
                "COD":'Sub-Saharan Africa',
                'CRI':'Latin America & Caribbean',
                'DOM':'Latin America & Caribbean',
                'HND':'Latin America & Caribbean',
                'LUX':"High income",
                "NIC":'Latin America & Caribbean',
                "PRY":'Latin America & Caribbean',
                "SLV" :'Europe & Central Asia',
                "SYR":'Middle East & North Africa',
                "URY":'Latin America & Caribbean',
                "VEN": 'Latin America & Caribbean',
                "XKX":'Europe & Central Asia'}))

In [0]:
# Imputation valeurs selon régions
data['IGEincome'] = data['IGEincome'].fillna(data["region"].map({'East Asia & Pacific':float(eap),
                'Europe & Central Asia':float(eca),
                'Latin America & Caribbean':float(lac),
                'Middle East & North Africa':float(mena),
                'South Asia':float(sa),
                'Sub-Saharan Africa':float(ssa)}))

In [0]:
data['IGEincome'] = data['IGEincome'].fillna(data.groupby('region')['IGEincome'].transform('mean'))

In [82]:
# recherche valeur nulles
print(pd.isnull(data).sum())

country            0
year_survey        0
quantile           0
nb_quantiles       0
income             0
gdpppp             0
gini               0
pop                0
region             0
year            1300
IGEincome          0
dtype: int64


# Elast 1.1


In [83]:
elast = data[["country","IGEincome"]]
elast = elast.groupby(["country"]).mean()
elast.sort_values(by=['IGEincome'], inplace=True, ascending=False)
elast = elast[elast['IGEincome']> 1]

elast.head(5)

,IGEincome
country,
COL,1.095440
ECU,1.029957
UGA,1.029195
GTM,1.015206


In [84]:
list_country = list(elast.index)
list_country

['COL', 'ECU', 'UGA', 'GTM']

In [0]:
for i in data["country"] : 
  if i in list_country :
    data.loc[data["country"]==i, "IGEincome"] = float('NaN')

In [86]:
# recherche valeur nulles
print(pd.isnull(data).sum())

country            0
year_survey        0
quantile           0
nb_quantiles       0
income             0
gdpppp             0
gini               0
pop                0
region             0
year            1300
IGEincome        400
dtype: int64


In [0]:
data['IGEincome'] = data['IGEincome'].fillna(data.groupby('region')['IGEincome'].transform('mean'))

In [88]:
# recherche valeur nulles
print(pd.isnull(data).sum())

country            0
year_survey        0
quantile           0
nb_quantiles       0
income             0
gdpppp             0
gini               0
pop                0
region             0
year            1300
IGEincome          0
dtype: int64


In [89]:
elast = data[["country","region","IGEincome"]]
elast = elast.groupby(["country","region"]).mean()
elast.sort_values(by=['IGEincome'], inplace=True, ascending=False)
elast = elast[elast['IGEincome']> 1]

elast.head(5)

,,IGEincome
country,region,


* Aucune valeur au dessus de 1.

# FJI

In [0]:
data.loc[data["country"]=="FJI", "gdpppp"] = float('NaN')

In [91]:
data.loc[data["country"]=="FJI", "gdpppp"]

3300   NaN
3301   NaN
3302   NaN
3303   NaN
3304   NaN
        ..
3395   NaN
3396   NaN
3397   NaN
3398   NaN
3399   NaN
Name: gdpppp, Length: 100, dtype: float64

In [0]:
data['gdpppp'] = data['gdpppp'].fillna(data.groupby('region')['gdpppp'].transform('mean'))

In [93]:
data.loc[data["country"]=="FJI", "gdpppp"]

3300    4401.21322
3301    4401.21322
3302    4401.21322
3303    4401.21322
3304    4401.21322
           ...    
3395    4401.21322
3396    4401.21322
3397    4401.21322
3398    4401.21322
3399    4401.21322
Name: gdpppp, Length: 100, dtype: float64

In [0]:
data["AVGincome"] = float('NaN')

In [0]:
data["AVGincome"] = data["AVGincome"].fillna(data.groupby('country')['income'].transform('mean'))

In [0]:
data.to_csv("/content/drive/My Drive/DATASETS/data_full_clean.csv", index=False)